In [1]:
!pip install langchain transformers datasets faiss-cpu sentence-transformers pypdf2 SentencePiece tabula-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 92.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 110.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.2/174.2 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.9

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/dlt_results

/content/drive/MyDrive/dlt_results


In [4]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain import FAISS
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, TFAutoModel, AutoModel
from datasets import Dataset
import faiss
import tabula
from utils import *
import pprint
import re

## Read PDF and parse tables

In [6]:
pdf_file = "./jpmc_annualreport-2022.pdf"
tables = tabula.read_pdf(pdf_file,pages="2")
print (f"Number of tables extracted, {len(tables)}")

Number of tables extracted, 1


In [8]:
tables[0]

,"(in millions, except per share, ratio data and headcount)",Unnamed: 0,2022,Unnamed: 1,2021,Unnamed: 2,2020
0,Selected income statement data,NaN,NaN,NaN,NaN,NaN,NaN
1,Total net revenue,$,"128,695",$,"121,649",$,"119,951"
2,Total noninterest expense,NaN,"76,140",NaN,"71,343",NaN,"66,656"
3,Pre-provision profit(a),NaN,"52,555",NaN,"50,306",NaN,"53,295"
4,Provision for credit losses,NaN,"6,389",NaN,"(9,256 )",NaN,"17,480"
5,Net income,$,"37,676",$,"48,334",$,"29,131"
6,Per common share data,NaN,NaN,NaN,NaN,NaN,NaN
7,Net income per share:,NaN,NaN,NaN,NaN,NaN,NaN
8,Basic,$,12.10,$,15.39,$,8.89
9,Diluted,NaN,12.09,NaN,15.36,NaN,8.88


In [38]:
table_text = ""
for table in tables:
  # Drop unnamed columns
  unnamed_columns_rename = {col:"" \
                            for col in table.columns if 'Unnamed' in col}
  table = table.dropna(axis=1, how='all').dropna(axis=0, how='all').rename(columns=unnamed_columns_rename)
  table_header = table.columns.tolist()
  for rows in table.iterrows():
    row = list(map(str,rows[1].tolist()))
    table_text += table_row_to_text(table_header, row)
  table_text += "<eos>"

In [39]:
unknown_characters = ['■', '◆', '□']
pattern = re.compile('|'.join(map(re.escape, unknown_characters)))
cleaned_table_text = re.sub(pattern, '', table_text)

In [40]:
len(cleaned_table_text)

9689

In [42]:
text_splitter = CharacterTextSplitter(
        separator=";",
        chunk_size=500,
        chunk_overlap=200,
        length_function=len
    )
chunks = text_splitter.split_text(cleaned_table_text)
print ("Number of chunks created",len(chunks))

Number of chunks created 32


## vectorise the chunks

In [43]:
model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [44]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [45]:
import datasets
print(datasets.__version__)

2.15.0


In [46]:
from datasets import Dataset, DatasetDict

# Assuming 'chunks' is a list of text chunks
data = {
    "text": chunks
}

# Create a Hugging Face dataset using DatasetDict
custom_dataset = DatasetDict({"train": Dataset.from_dict(data)})

# Map the "embeddings" column
def map_embeddings(example):
    embeddings = get_embeddings(example["text"]).detach().numpy()[0]
    return {"embeddings": embeddings}

embeddings_dataset = custom_dataset["train"].map(map_embeddings)


Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [17]:
# data = {
#     "text": chunks
# }
# # Create a Hugging Face dataset
# custom_dataset = Dataset.from_dict(data)
# # Map the "embeddings" column
# embeddings_dataset = custom_dataset.map(
#     lambda x: {"embeddings": get_embeddings(x["text"]).detach().numpy()[0]}
# )

In [47]:
embeddings_dataset.add_faiss_index(column="embeddings", index_name="faiss_index")

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['text', 'embeddings'],
    num_rows: 32
})

## Search relevant chunk with question

In [48]:
question = "What is the percentage of interest expense to net income in 2019 Q2"
question_embedding = get_embeddings([question]).detach().numpy()
question_embedding.shape

(1, 768)

In [24]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "faiss_index", question_embedding, k=5
)

In [25]:
samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)
samples_df

,text,embeddings,scores
4,the Net income of Unnamed: 2 is $ ; the Net in...,"[-0.30926313996315, 0.006089046597480774, -0.3...",37.623306
3,the Net income per share: of Unnamed: 2 is nan...,"[-0.13005410134792328, -0.20198948681354523, -...",37.361641
2,"the Total noninterest expense of 2022 is 76,14...","[-0.08556223660707474, -0.16636861860752106, -...",36.744488
1,"(in millions, except per share, ratio data and...","[-0.21795296669006348, 0.10966174304485321, -0...",34.846066
0,"(in millions, except per share, ratio data and...","[-0.10045704990625381, 0.026219500228762627, -...",33.419678


# Inference

In [53]:
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast
import torch
TOKENIZER = T5Tokenizer.from_pretrained("t5-base")
MODEL_finetune = T5ForConditionalGeneration.from_pretrained("./finqa_finetune_t5.pth/")
MODEL_normal = T5ForConditionalGeneration.from_pretrained("t5-small")

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [27]:
MODEL.eval();

In [30]:
# for row in samples_df.iterrows():
#     context_i = "context: " + row[1]["text"]
#     question_context_tokens = TOKENIZER(question,context_i, max_length=1024, padding="max_length",
#                                                     truncation=True, pad_to_max_length=True, add_special_tokens=True)
#     input_ids = torch.tensor(question_context_tokens["input_ids"], dtype=torch.long).view(1,-1)
#     print(input_ids.shape)
#     with torch.no_grad():
#         generated_output = MODEL.generate(
#             input_ids=input_ids,
#             max_length=128,  # Set the desired maximum length for generated text
#             num_beams=10,    # You can adjust the number of beams for beam search
#             temperature=0.1  # You can adjust the temperature for sampling,
#             )
#         generated_text = TOKENIZER.decode([x for x in generated_output[0] if x!= -100], skip_special_tokens=True)
#         print ("-"*10)
#         pp = pprint.PrettyPrinter(width=40)
#         pp.pprint(generated_text)
#         # print(generated_text)
#         print ("-"*10)

torch.Size([1, 1024])
----------
('Step 1: Add 2 and 2. This gives the '
 'result: 2 ####### Step 2: Add 2 and '
 '2. This gives the result: 2 ####### '
 'Step 3: Add 2 and 2. This gives the '
 'result: 2 ####### Step 4: Add 2 and '
 '2. This gives the result: 2 ####### '
 'Step 5: Add 2 and 2. This gives the '
 'result: 2 ####### Step 5: Add 2 and '
 '2. This gives the result: 2 ####### '
 'Step 5: Add 2 and 2. This gives the '
 'result: 2')
----------
torch.Size([1, 1024])
----------
('Step 1: Add 2 and 2. This gives the '
 'result: 2 ####### Step 2: Add 2 and '
 '2. This gives the result: 2 ####### '
 'Step 3: Add 2 and 2. This gives the '
 'result: 2 ####### Step 4: Add 2 and '
 '2. This gives the result: 2 ####### '
 'Step 5: Add 2 and 2. This gives the '
 'result: 2 ####### Step 5: Add 2 and '
 '2. This gives the result: 2 ####### '
 'Step 5: Add 2 and 2. This gives the '
 'result: 3')
----------
torch.Size([1, 1024])
----------
('Step 1: Add 76 and 64. This gives '
 'the result:

In [65]:
def search_question(ques,k=5, verbose = True):
  question_embedding = get_embeddings([question]).detach().numpy()
  scores, samples = embeddings_dataset.get_nearest_examples(
    "faiss_index", question_embedding, k=k)

  samples_df = pd.DataFrame.from_dict(samples)
  samples_df["scores"] = scores
  samples_df.sort_values("scores", ascending=False, inplace=True)
  if verbose:
    display(samples_df)
  for model_key,m in {"Finetune":MODEL_finetune, "Normal": MODEL_normal}.items():
    print('\n---------',model_key,'----------')
    m.eval();
    for row in samples_df.iterrows():
      context_i = "context: " + row[1]["text"]
      question_context_tokens = TOKENIZER("question: "+question,context_i, max_length=1024, padding="max_length",
                                                      truncation=True, pad_to_max_length=True, add_special_tokens=True)
      input_ids = torch.tensor(question_context_tokens["input_ids"], dtype=torch.long).view(1,-1)
      # print(input_ids.shape)
      with torch.no_grad():
          generated_output = m.generate(
              input_ids=input_ids,
              max_length=128,  # Set the desired maximum length for generated text
              num_beams=10,    # You can adjust the number of beams for beam search
              temperature=0.1,do_sample=True, max_new_tokens = 50  # You can adjust the temperature for sampling,
              )
          generated_text = TOKENIZER.decode([x for x in generated_output[0] if x!= -100], skip_special_tokens=True)
          print ("-"*80)
          pp = pprint.PrettyPrinter(width=80)
          pp.pprint(generated_text)
  return samples_df

In [66]:
context_df = search_question("What is the ratio of Net Income in 2022 to net income in 2021",k=5)

,text,embeddings,scores
4,"the Net income of 2022 is 37,676 ; the Net inc...","[-0.2652001678943634, -0.06299582123756409, -0...",38.359867
3,"(in millions, except per share, ratio data and...","[-0.11729767173528671, 0.13592670857906342, -0...",36.750519
2,"the Provision for credit losses of 2022 is 6,3...","[-0.26518625020980835, -0.15498270094394684, -...",36.660255
1,"the Total net revenue of 2020 is 119,951 ;(in ...","[-0.19604815542697906, -0.12280641496181488, -...",35.862976
0,the Selected income statement data of 2020 is ...,"[-0.22313688695430756, 0.07183703035116196, -0...",35.652794



--------- Finetune ----------


Both `max_new_tokens` (=50) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


--------------------------------------------------------------------------------
'Step 1: Divide 22.6 by 48.3. This gives the result: 68.4%'


Both `max_new_tokens` (=50) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

--------------------------------------------------------------------------------
'29,131'


Both `max_new_tokens` (=50) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


--------------------------------------------------------------------------------
'66,656'


Both `max_new_tokens` (=50) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


--------------------------------------------------------------------------------
'76,140'


In [73]:
context_df["text"][4]

'the Net income of 2022 is 37,676 ; the Net income of is $ ; the Net income of 2021 is 48,334 ; the Net income of is $ ; the Net income of 2020 is 29,131 ;(in millions, except per share, ratio data and headcount) the Per common share data of is nan ; the Per common share data of 2022 is nan ; the Per common share data of is nan ; the Per common share data of 2021 is nan ; the Per common share data of is nan ; the Per common share data of 2020 is nan'

In [74]:
pp.pprint(context_df["text"][4])

('the Net income of 2022 is 37,676 ; '
 'the Net income of is $ ; the Net '
 'income of 2021 is 48,334 ; the Net '
 'income of is $ ; the Net income of '
 '2020 is 29,131 ;(in millions, except '
 'per share, ratio data and headcount) '
 'the Per common share data of is nan '
 '; the Per common share data of 2022 '
 'is nan ; the Per common share data '
 'of is nan ; the Per common share '
 'data of 2021 is nan ; the Per common '
 'share data of is nan ; the Per '
 'common share data of 2020 is nan')


In [57]:
search_question("What is the Net Income in 2022")

,text,embeddings,scores
4,"the Net income of 2022 is 37,676 ; the Net inc...","[-0.2652001678943634, -0.06299582123756409, -0...",38.359867
3,"(in millions, except per share, ratio data and...","[-0.11729767173528671, 0.13592670857906342, -0...",36.750519
2,"the Provision for credit losses of 2022 is 6,3...","[-0.26518625020980835, -0.15498270094394684, -...",36.660255
1,"the Total net revenue of 2020 is 119,951 ;(in ...","[-0.19604815542697906, -0.12280641496181488, -...",35.862976
0,the Selected income statement data of 2020 is ...,"[-0.22313688695430756, 0.07183703035116196, -0...",35.652794


--------- Finetune ----------


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Both `max_new_tokens` (=50) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


--------------------------------------------------------------------------------
('Step 1: Divide 334 by 37,676. This gives the result: 1.075 ####### Step 2: '
 'Multiply 1.075 by const_100. This gives the result: 1.075')


Both `max_new_tokens` (=50) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

--------------------------------------------------------------------------------
'29,131'


Both `max_new_tokens` (=50) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


--------------------------------------------------------------------------------
('66,656 ;(in millions, except per share, ratio data and headcount) the '
 'Pre-provision profit(a) of is nan')


Both `max_new_tokens` (=50) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


--------------------------------------------------------------------------------
'76,140'
